In [2]:
!pip install decorator==5.0.9 

  Attempting uninstall: decorator
    Found existing installation: decorator 5.0.6
    Uninstalling decorator-5.0.6:
      Successfully uninstalled decorator-5.0.6


You should consider upgrading via the 'c:\users\pajag\anaconda3\python.exe -m pip install --upgrade pip' command.


In [7]:
import numpy as np
import networkx as nx
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

social_graph = nx.erdos_renyi_graph(100, 0.1)

for node in social_graph.nodes:
    profile = f"User {node}: "
    interests = ", ".join(np.random.choice(["music", "sports", "movies", "travel"], size=np.random.randint(1, 4)))
    comments = " ".join(np.random.choice(["I love this!", "This is interesting.", "Not my cup of tea."], size=np.random.randint(1, 4)))
    sentiment = np.random.choice(["positive", "neutral", "negative"])
    text_data = f"{profile} Interests: {interests}. Comments: {comments}. Sentiment: {sentiment}"
    social_graph.nodes[node]['text'] = text_data

texts = [social_graph.nodes[node]['text'] for node in social_graph.nodes]
labels = [social_graph.nodes[node]['text'].split()[-1] for node in social_graph.nodes]

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(max_words, 32, input_length=max_len))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')


Epoch 1/5
2/2 [==============================] - 2s 471ms/step - loss: 0.6938 - accuracy: 0.1562 - val_loss: 0.6761 - val_accuracy: 0.2500
Epoch 2/5
2/2 [==============================] - 0s 40ms/step - loss: 0.6749 - accuracy: 0.4062 - val_loss: 0.6545 - val_accuracy: 0.2500
Epoch 3/5
2/2 [==============================] - 0s 39ms/step - loss: 0.6541 - accuracy: 0.4062 - val_loss: 0.6313 - val_accuracy: 0.2500
Epoch 4/5
2/2 [==============================] - 0s 41ms/step - loss: 0.6354 - accuracy: 0.4062 - val_loss: 0.6046 - val_accuracy: 0.2500
Epoch 5/5
1/1 [==============================] - 0s 21ms/step - loss: 0.5853 - accuracy: 0.6000
Test accuracy: 0.6000000238418579


In [10]:
def preprocess_input(user_input):
    user_input = [user_input]
    sequences = tokenizer.texts_to_sequences(user_input)
    X_input = pad_sequences(sequences, maxlen=max_len)
    return X_input

def predict_sentiment(input_text):
    X_input = preprocess_input(input_text)
    prediction = model.predict(X_input)
    sentiment_label = "positive" if prediction > 0.5 else "negative"
    return sentiment_label, prediction[0][0]

user_input = input("Enter a text to analyze sentiment: ")

sentiment, confidence = predict_sentiment(user_input)
print(f"Predicted sentiment: {sentiment} with confidence {confidence}")


Enter a text to analyze sentiment: ok
Predicted sentiment: positive with confidence 0.554423987865448
